In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks'
file_path = '/content/drive/My Drive/Colab Notebooks/23Fall/spotify52kData.csv'
df = pd.read_csv(file_path)
df.head()

/content/drive/My Drive/Colab Notebooks


,songNumber,artists,album_name,track_name,popularity,duration,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve

In [21]:
feature_col = ['duration','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']

y = df['popularity'].values

In [20]:
RMSE = []
betas = []
alphas = []

for f in feature_col:
  X = df[f].values.reshape(-1, 1)

  # train test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # hyperparameter tuning
  param_grid = {'alpha': [1e-2, 1e-1, 1, 10, 100, 1000]}
  grid_search = GridSearchCV(Ridge(), param_grid, scoring='neg_mean_squared_error', cv=10)
  grid_search.fit(X_train, y_train)
  best_alpha = grid_search.best_params_['alpha']
  best_model = grid_search.best_estimator_
  beta = best_model.coef_[0]

  # evaluate the model
  pred = best_model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, pred))

  RMSE.append(rmse)
  betas.append(beta)
  alphas.append(best_alpha)

# print results
results4 = pd.DataFrame({'feature': feature_col, 'RMSE': RMSE, 'coef': betas, 'best hyper': alphas})
results4

,feature,RMSE,coef,best hyper
0,duration,21.645897,-0.000010,0.01
1,danceability,21.665287,4.281570,10.00
2,energy,21.638446,-4.579709,10.00
3,loudness,21.655747,0.271674,1000.00
4,speechiness,21.664300,-8.065406,10.00
5,acousticness,21.674434,1.572007,100.00
6,instrumentalness,21.456623,-9.642834,10.00
7,liveness,21.671883,-5.206433,10.00
8,valence,21.665546,-2.716831,100.00
9,tempo,21.686514,-0.001560,1000.00


In [23]:
X = df[feature_col].values

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# hyperparameter tuning
param_grid = {'alpha': [1e-2, 1e-1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(Ridge(), param_grid, scoring='neg_mean_squared_error', cv=10)
grid_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_['alpha']
best_model = grid_search.best_estimator_
beta = best_model.coef_

# evaluate the model
pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))

# print results
print(f'RMSE: {rmse}, best hyper: {best_alpha}')
results5 = pd.DataFrame({'feature': feature_col, 'coef': betas})
results5

RMSE: 21.111876225733532, best hyper: 10


,feature,coef
0,duration,-0.000010
1,danceability,4.281570
2,energy,-4.579709
3,loudness,0.271674
4,speechiness,-8.065406
5,acousticness,1.572007
6,instrumentalness,-9.642834
7,liveness,-5.206433
8,valence,-2.716831
9,tempo,-0.001560
